In [1]:
import polars as pl
import pandas as pd
from utils import *

In [2]:
data_folder = '../data/'
train_folder = data_folder + '/train/'
test_folder = data_folder + '/test/'


# Get IDs of tweets used for train/test

In [3]:
positive = data_folder + 'final_database/positive.parquet'

positive = pl.read_parquet(positive)

pos_ids = positive['id'].to_list()

In [4]:
trainfile = data_folder + 'train/train.csv'
testfolder = '../data/test/'

train = pd.read_csv(trainfile)

train_ids = train['id'].to_list()

In [5]:
testfile = testfolder + 'test_raw.csv'

test = pd.read_csv(testfile)

test_ids = test['id'].to_list()

In [6]:
testunlabelfile = testfolder + 'test_unlabel.csv'

testunlabel = pd.read_csv(testunlabelfile)

testunlabel_ids = testunlabel['id'].to_list()

In [7]:
ids_to_delete = pos_ids + train_ids + test_ids + testunlabel_ids

# Get unlabeled data

In [8]:
unlabel_filename = data_folder + 'final_database/combined_search.parquet'
unlabel = pl.read_parquet(unlabel_filename).with_columns([
    pl.col("created_at").str.strptime(pl.Datetime,
                                      format="%Y-%m-%dT%H:%M:%S.%fZ", strict=False)
]).filter(pl.col("created_at") < pl.datetime(2023, 1, 1)).to_pandas()
len(unlabel)

/tmp/ipykernel_18313/438098856.py:3: ChronoFormatWarning: Detected the pattern `.%f` in the chrono format string. This pattern should not be used to parse values after a decimal point. Use `%.f` instead. See the full specification: https://docs.rs/chrono/latest/chrono/format/strftime
  pl.col("created_at").str.strptime(pl.Datetime,


309967

In [9]:
unlabel = unlabel[~unlabel['id'].isin(ids_to_delete)]

len(unlabel)

304078

In [10]:
unlabel = delete_tweets(unlabel)

After removing Fogo Cruzado tweets: (250716, 46)
After removing duplicates texts: (240211, 46)
After removing retweets: (240204, 46)
After removing replies: (199122, 46)
After removing is temporarily unavailable: (199050, 46)
After removing @fogocruzadoapp: (199020, 46)
After removing fogocruzado: (199020, 46)
After removing #FogoCruzadoRJ: (199015, 46)


In [11]:
unlabel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199015 entries, 0 to 309966
Data columns (total 46 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   author_id                        199015 non-null  object        
 1   conversation_id                  199015 non-null  object        
 2   created_at                       199015 non-null  datetime64[ns]
 3   edit_history_tweet_ids           199015 non-null  object        
 4   id                               199015 non-null  object        
 5   in_reply_to_user_id              0 non-null       object        
 6   lang                             199015 non-null  object        
 7   possibly_sensitive               199015 non-null  bool          
 8   referenced_tweets                16142 non-null   object        
 9   text                             199015 non-null  object        
 10  entities.mentions                3700 non-nu

In [12]:
unlabel.text = unlabel['text'].apply(cleantxt)
unlabel[['id','text']].sample(10)

,id,text
178540,922815644546076673,Pqp q tiro URL
102777,1267311107749928960,Mostro oque Eu sou na pratica e sou transparen...
185909,938491689303465984,all that matters eh tiro no cu
151097,866775517403516930,Já viu alguém na tropa morrer com tiro nas cos...
169602,905428423241289728,essa música do imaginasamba com a Simone e Sim...
25948,1513525816390414336,O problema parceiro é que um tiro de 762 faz...
223111,1012675703647633408,Olá senhores tivemos que apagar a postagem do...
17197,1543597416565309440,Menor mas troca tiro que eu conheci descanse ...
89256,1307314570147639299,Oi USR você está a par que existem outras colo...
270027,1116037284032335873,PRAÇA SECA Ônibus da Viação Novacap precisou ...


In [13]:
unlabel[['id','text']].to_csv(data_folder + 'train/unlabel.csv', index=False)